Accelerated Bayesian Causal Forest
https://johaupt.github.io/blog/xbcf.html

Accelerated Bayesian Causal Forest (XBCF) to estimate the conditional average treatment effect (or uplift) using a specialized version of Bayesian Additive Regression Trees (BART). It’s better described as Bayesian boosted trees for non-parametric causal inference.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xbcausalforest import XBCF

## BPIC 2017 dataset

In [2]:
df = pd.read_csv("bpi2017_final.csv")

In [3]:
feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

In [4]:
# Split data to training and testing samples for model validation (next section)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11101)

In [5]:
treatment=df_train['treatmentOffer']
X = df_train[feature_names]
y=df_train['offerSuccess']

treatment_test=df_test['treatmentOffer']
X_test = df_test[feature_names]
y_test=df_test['offerSuccess']

In [ ]:
treated_df = 

# df1 = df[df['treatment'] == 1]
# df2 = df[df['treatment'] == 0]

In [6]:
NUM_TREES_PR  = 200
NUM_TREES_TRT = 100

cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=50, 
    burnin=15,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=NUM_TREES_TRT,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(y)/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(y)/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    standardize_target=True, # standardize y and unstandardize for prediction
         )

Since we specify the model as a sum of two BARTs, we can pass different sets of covariates to the outcome model and the treatment model, denoted by x and x_t. z is the treatment indicator coded 0/1.

In [ ]:
## should this be included in the second X, as the propensity score
p_model = ElasticNetPropensityModel()
p = p_model.fit_predict(X, treatment)
print(p)

In [7]:
%%time
cf.fit(
    x_t=X, # Covariates treatment effect
    x=X, # Covariates outcome (including propensity score)
    y=y,  # Outcome
    z=treatment, # Treatment group
)

CPU times: user 10h 56min 59s, sys: 10min 8s, total: 11h 7min 7s
Wall time: 1h 21min 21s


XBCF(num_sweeps = 50, burnin = 15, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 19, mtry_trt = 19, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.0007494867168461104, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 100, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 0.00024982890561537014, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [8]:
%%time
tau_xbcf = cf.predict(X_test, return_mean=True)
tau_xbcf

CPU times: user 29.5 s, sys: 186 ms, total: 29.7 s
Wall time: 29.9 s


array([ 0.03308522, -0.0446359 , -0.05176309, ..., -0.03810597,
        0.0370041 , -0.03629823])

In [9]:
# Calculate statistics
data = np.reshape(tau_xbcf, -1)
minimum = np.min(data)
first_quartile = np.percentile(data, 25)
median = np.median(data)
third_quartile = np.percentile(data, 75)
maximum = np.max(data)

# Interquartile range (IQR)
iqr = third_quartile - first_quartile

# Define upper and lower bounds for outliers
upper_bound = third_quartile + 1.5 * iqr
lower_bound = first_quartile - 1.5 * iqr

# Detect outliers
outliers = data[(data < lower_bound) | (data > upper_bound)]

# Print the statistics
print("Minimum:", minimum)
print("First Quartile:", first_quartile)
print("Median:", median)
print("Third Quartile:", third_quartile)
print("Maximum:", maximum)
print("Interquartile Range:", iqr)
print("Upper Bound (Outliers):", upper_bound)
print("Lower Bound (Outliers):", lower_bound)
print("Outliers:", outliers)

ite_bart = [minimum, first_quartile, median, third_quartile, maximum, iqr, upper_bound, lower_bound]

Minimum: -0.9658506333554799
First Quartile: -0.04886845276852485
Median: -0.0368942777357658
Third Quartile: 0.03376068109580269
Maximum: 2.5479356161410225
Interquartile Range: 0.08262913386432755
Upper Bound (Outliers): 0.15770438189229402
Lower Bound (Outliers): -0.17281215356501617
Outliers: [-0.7247538  -0.22329774  0.77603805 ...  0.17415126  2.38680562
 -0.40932287]


In [ ]:
%store -r df_results
lib = "xbcausalforest"
method = "Accelerated Bayesian Causal Forest"
ite = ite_bart
ate = tau_xbcf.mean()

if method in df_results['method'].values:
     # If the method is already in the DataFrame, update the ATE and ITE columns
    df_results.loc[df_results['method'] == method, 'ATE'] = ate
    index = df_results[df_results['method'] == method].index[0]
    df_results.at[index, 'ITE'] = ite
else:
    # If the method is not in the DataFrame, add a new row
    df_results = df_results._append({'method': method, 'ATE': ate, 'ITE': ite, 'Library': lib}, ignore_index=True)

print(df_results)
%store df_results

## Synthetic Dataset

In [2]:
df_synth = pd.read_csv("synthetic_dataset.csv")
df_synth.head()
synthetic_features = ['NumberOfOffers', 'concept:name',
       'lifecycle:transition', 'time:timestamp', 'elementId', 'resourceId',
       'weekdayApplication', 'timeApplication']

In [3]:
# Split data to training and testing samples for model validation (next section)
df_synth_train, df_synth_test = train_test_split(df_synth, test_size=0.2, random_state=11101)

synth_treatment=df_synth_train['treatment']
synth_X = df_synth_train[synthetic_features]
synth_y=df_synth_train['treatmentSuccess']

synth_treatment_test=df_synth_test['treatment']
synth_X_test = df_synth_test[synthetic_features]
synth_y_test=df_synth_test['treatmentSuccess']

In [4]:
NUM_TREES_PR  = 200
NUM_TREES_TRT = 100

cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=50, 
    burnin=15,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=NUM_TREES_TRT,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(synth_y)/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(synth_y)/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    standardize_target=True, # standardize y and unstandardize for prediction
         )

In [5]:
%%time
cf.fit(
    x_t=synth_X, # Covariates treatment effect
    x=synth_X, # Covariates outcome (including propensity score)
    y=synth_y,  # Outcome
    z=synth_treatment, # Treatment group
)

CPU times: user 21min 50s, sys: 11.6 s, total: 22min 2s
Wall time: 6min 4s


XBCF(num_sweeps = 50, burnin = 15, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 8, mtry_trt = 8, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.002735641295552384, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 100, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 0.0009118804318507947, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [6]:
%%time
synth_tau_xbcf = cf.predict(synth_X_test, return_mean=True)
synth_tau_xbcf

CPU times: user 2.57 s, sys: 12 ms, total: 2.58 s
Wall time: 2.56 s


array([ 2.00000441, -0.01259462,  2.00000071, ...,  1.99999129,
        2.00000158,  2.00000441])

In [7]:
import evaluation_metrics
true_ate = 2
boxplot = evaluation_metrics.boxplot_ite(synth_tau_xbcf)
metrics = evaluation_metrics.evaluation_metrics(true_ate, synth_tau_xbcf)
print(boxplot, metrics)

[-0.04994711611560761, 1.999994124582845, 2.0000020060979793, 2.0000037880084416, 2.0000082955627065, 9.6634255966066e-06, 2.0000182831468365, 1.99997962944445] [0.5152341866786688, 0.2577259784442367, 0.7177981517659884, 0.2577223132045925, 0.3588990758829942]


In [8]:
%store -r df_synthetic_results_metric

method = "Accelerated Bayesian Causal Forest"
ate = synth_tau_xbcf.mean()
ite = boxplot
metric = metrics

df_synthetic_results_metric = df_synthetic_results_metric._append({'method': method, 'ATE': ate, 'ITE': ite, 'metrics': metric}, ignore_index=True)

print(df_synthetic_results_metric)
%store df_synthetic_results_metric

                                           method  \
0                                    S-Learner LR   
1                                   XGBTRegressor   
2                              BaseTRegressor XGB   
3                               BaseTRegressor LR   
4                              BaseXRegressor XGB   
5                               BaseXRegressor LR   
6   BaseXRegressor XGB (without propensity score)   
7    BaseXRegressor LR (without propensity score)   
8                              BaseRRegressor XGB   
9                               BaseRRegressor LR   
10        BaseRRegressor XGB (with random weight)   
11  BaseRRegressor XGB (without propensity score)   
12                           Neural Network (MLP)   
13                                         BCAUSS   
14                                          CEVAE   
15             Accelerated Bayesian Causal Forest   

                                                  ITE       ATE  \
0   [1.999999999999998, 1.99999